In [ ]:
from elasticsearch import Elasticsearch
import os
import json
from tqdm.notebook import tqdm

In [ ]:
es = Elasticsearch(
    hosts=os.environ["ELASTIC_HOST"],
    http_auth=(os.environ["ELASTIC_USERNAME"], os.environ["ELASTIC_PASSWORD"]),
)

In [ ]:
data_path = "../data/works.json"

In [ ]:
n_lines = sum(1 for line in open(data_path))
n_lines

In [ ]:
def get_lines():
    with open(data_path) as f:
        for line in f:
            yield json.loads(line)
            
lines = get_lines()

In [ ]:
subjects = [
    subject['label'] 
    for line in tqdm(get_lines(), total=n_lines)
    for subject in line['subjects']
]

In [ ]:
from collections import Counter

subject_count = Counter(subjects)

In [ ]:
len(subject_count)

In [ ]:
common_subjects = subject_count.most_common(100_000)

In [ ]:
a = 0
variants = {}

for subject, count in tqdm(common_subjects):
    response = es.search(
        index="variant-names",
        body={
            "query":{
                "term": {
                    "variant_names.keyword": subject
                }
            }
        }
    )
    if response['hits']['total']['value'] > 0:
        variants[subject] = response['hits']['hits'][0]['_source']['variant_names']
        a += 1
    else:
        variants[subject] = []

In [ ]:
a

# assessing overlap

In [ ]:
blah = Counter()

In [ ]:
overlap_counts = Counter()

for subject, count in tqdm(common_subjects):
    response = es.search(
        index="variant-names",
        body={
            "query":{
                "term": {
                    "variant_names.keyword": subject
                }
            }
        }
    )
    if response['hits']['total']['value'] > 0:
        overlap_counts[
            hash(str(response['hits']['hits'][0]['_source']['variant_names']))
        ] += 1

In [ ]:
total_overlap = 0
n_overlap = 0

for i, count in overlap_counts.most_common():
    if count > 1:
        total_overlap += count
        n_overlap += 1

total_overlap, n_overlap